In [1]:
import os
import sys
import yaml
from copy import deepcopy
from typing import (
    Dict,
    List, 
    Optional,
    Tuple,
    Union
)

In [2]:
__file__ = os.path.join(os.getcwd(),"cs_wrap1.ipynb")

In [3]:
mod_path = os.path.join(os.path.dirname(os.path.abspath(__file__)),"..")

In [4]:
sys.path.append(mod_path)

In [5]:
from convert_source.cs_utils.bids_info import (
    construct_bids_dict,
    construct_bids_name,
    search_bids
)

In [85]:
from convert_source.cs_utils.utils import (
    BIDSimg,
    collect_info,
    convert_image_data,
    dict_multi_update,
    get_metadata,
    list_in_substr,
    SubInfoError,
    SubDataInfo,
    read_json,
    write_josn,
    # new functions
    comp_dict,
    depth,
    list_dict
)

In [86]:
from convert_source.cs_utils.fileio import(
    LogFile,
    TmpDir,
    ConversionError
)

In [8]:
from convert_source.cs_utils.const import(
    DEFAULT_CONFIG,
    BIDS_PARAM
)

In [9]:
from convert_source.imgio.dcmio import (
    is_valid_dcm
)

In [10]:
# test_config:
test_config = os.path.abspath(
    os.path.join(
        __file__,
        '..',
        '..',
        'notebooks.old',
        'config.test',
        'config.default.yml'
        ))
test_config

'/Users/adebayobraimah/Desktop/projects/convert_source/notebooks.old/config.test/config.default.yml'

In [11]:
def read_config(config_file: Optional[str] = "", 
                verbose: Optional[bool] = False
                ) -> Tuple[Dict[str,str],Dict,Dict,Dict,List[str]]:
    '''Reads configuration file and creates a dictionary of search terms for 
    each modality provided that each BIDS modality is used as a key via the 
    keyword 'modality_search'. Should BIDS related parameter descriptions need 
    to be used when renaming files, the related search and mapping terms can included 
    via the keywords 'bids_search' and 'bids_map', respectively. If these keywords 
    are not specified, then empty dictionaries are returned. Should exclusions be provided 
    (via the key 'exclude') then an exclusion list is created. Should this not be provided, 
    then an empty list is returned.

    BIDS modalities:
        - anat:
            - T1w, T2w, FLAIR, etc.
        - func:
            - bold
                - task:
                    - resting state, <task-name>
        - dwi
        - fmap

    Usage example:
        >>> [search_dict, bids_search, bids_map, meta_dict, exclusion_list] = read_config(config_file)
    
    Arguments:
        config_file: File path to yaml configuration file. If no file is used, then the default configuration file is used.
        verbose: Prints additional information to screen.
    
    Returns: 
        Tuple of dictionaries and a list that consists of:
            * search_dict: Nested dictionary of heuristic modality search terms for BIDS modalities.
            * bids_search: Nested dictionary of heuristic BIDS search terms.
            * bids_map: Corresponding nested dictionary of BIDS mapping terms to rename files to.
            * meta_dict: Nested dictionary of metadata terms to write to JSON file(s).
            * exclusion_list: List of exclusion terms.
    
    Raises:
        ConfigFileReadError: Error that arises if no heuristic search terms are provided.
    '''
    class ConfigFileReadError(Exception):
        pass

    if config_file:
        config_file: str = os.path.abspath(config_file)
    else:
        config_file: str = DEFAULT_CONFIG

    with open(config_file) as file:
        data_map: Dict[str,str] = yaml.safe_load(file)
        if verbose:
            print("Initialized parameters from configuration file")
    
    # Required modality search terms
    if any("modality_search" in data_map for element in data_map):
        if verbose:
            print("Categorizing search terms")
        search_dict: Dict[str,str] = data_map["modality_search"]
        del data_map["modality_search"]
    else:
        if verbose:
            print("Heuristic search terms required. Exiting...")
        raise ConfigFileReadError("Heuristic search terms required. Exiting...")
    
    # BIDS search terms
    if any("bids_search" in data_map for element in data_map):
        if verbose:
            print("Including BIDS related search term settings")
        bids_search: Dict[str,str] = data_map["bids_search"]
        del data_map["bids_search"]
    else:
        if verbose:
            print("No BIDS related search term settings")
        meta_dict: Dict = dict()
    
    # BIDS mapping terms
    if any("bids_map" in data_map for element in data_map):
        if verbose:
            print("Corresponding BIDS mapping settings")
        bids_map: Dict[str,str] = data_map["bids_map"]
        del data_map["bids_map"]
    else:
        if verbose:
            print("No BIDS mapping settings")
        meta_dict: Dict = dict()
    
    # Metadata terms
    if any("metadata" in data_map for element in data_map):
        if verbose:
            print("Including additional settings for metadata")
        meta_dict: Dict[str,Union[str,int]] = data_map["metadata"]
        del data_map["metadata"]
    else:
        if verbose:
            print("No metadata settings")
        meta_dict: Dict = dict()
    
    # Exclusion terms  
    if any("exclude" in data_map for element in data_map):
        if verbose:
            print("Exclusion option implemented")
        exclusion_list: List[str] = data_map["exclude"]
        del data_map["exclude"]
    else:
        if verbose:
            print("Exclusion option not implemented")
        exclusion_list: List = list()
        
    return (search_dict,
            bids_search,
            bids_map,
            meta_dict,
            exclusion_list)

In [49]:
[search_dict,bids_search,bids_map,meta_dict,exclusion_list] = read_config(config_file=test_config)

In [13]:
# search_dict
# bids_search
# bids_map
# meta_dict
# exclusion_list

In [14]:
# Check BIDS search and map dicts
comp_dict(d1=bids_search,d2=bids_map)

True

In [15]:
# check depth of dicts

In [16]:
depth(search_dict['func'])

3

In [17]:
depth(search_dict['anat'])

2

In [18]:
BIDS_PARAM

{'info': {'sub': '', 'ses': ''},
 'anat': {'acq': '', 'ce': '', 'rec': '', 'run': '', 'modality_label': ''},
 'func': {'task': '',
  'acq': '',
  'ce': '',
  'dir': '',
  'rec': '',
  'run': '',
  'echo': '',
  'modality_label': ''},
 'dwi': {'acq': '', 'dir': '', 'run': '', 'modality_label': ''},
 'fmap': {'acq': '',
  'run': '',
  'case1': {'phasediff': '', 'magnitude1': '', 'magnitude2': ''},
  'case2': {'phase1': '', 'phase2': '', 'magnitude1': '', 'magnitude2': ''},
  'case3': {'magnitude': '', 'fieldmap': ''},
  'case4': {'ce': '', 'dir': '', 'modality_label': 'epi'}}}

In [19]:
meta_dict

{'common': {'Manufacturer': 'Philips',
  'ManufacturersModelName': 'Ingenia',
  'MagneticFieldStrength': 3,
  'InstitutionName': "Cincinnati Children's Hospital Medical Center"},
 'func': {'rest': {'ParallelAcquisitionTechnique': 'SENSE',
   'PhaseEncodingDirection': 'j',
   'MultibandAccelerationFactor': 6,
   'TaskName': 'Resting State',
   'dir': 'PA',
   'NumberOfVolumesDiscardedByScanner': 4}}}

In [20]:
# 1. read config file
# 2. check bids search and map dicts
# 3. collect subject directory information [sub/ses ID, and data paths], exclude data as necessary
# 4. pass to function: List[SubDataInfo], search_dict, bids_search, bids_map, and meta_dict

In [21]:
par_img_dir = "test.img"

In [22]:
par_img_dir = os.path.abspath(par_img_dir)

In [23]:
# 3 [exclusion list is used here]
subs_data = collect_info(parent_dir=par_img_dir,
                        exclusion_list=exclusion_list)

In [24]:
subs_data[90].ses

'001'

In [25]:
# TODO:
#   - Write function that takes input (of array) of SubDataInfo object(s) {sub,data,ses}, and search dict - to search and categorize data
#   - helper/support function to aid in search and categorization of data.

In [84]:
def bids_id(s:str,
            search_dict: Dict,
            bids_search: Optional[Dict] = None,
            bids_map: Optional[Dict] = None,
            bids_name_dict: Optional[Dict] = None
           ) -> Tuple[Dict[str,str],str,str,str]:
    '''
    TODO:
        * split the input string so that only the relevant parts of the filename are searched.
        * contingency function to search image file headers.
        
    Performs identification of descriptive BIDS information relevant for file naming, provided
    a BIDS search dictionary and a BIDS map dictionary. The resulting information is then placed
    in nested dictionary of BIDS related descriptive terms.
    
    Usage example:
        >>> bids_example_dict = bids_id("ImageFile00001.dcm")
        
    Arguments:
        s: Input str/file to be searched.
        search_dict: Dictionary of modality specific search terms.
        bids_search: Dictionary of BIDS specific search terms.
        bids_map: Dictionary of BIDS related terms to be mapped to BIDS search terms.
        bids_name_dict: Existing BIDS name dictionary. If provided, an updated copy of this dictionary is returned.
        
    Returns:
        Tuple that consists of:
            * Nested dictionary of BIDS descriptive naming related terms.
            * Modality type.
            * Modality label.
            * Task label.
    '''
    search_arr: List[str] = list_dict(d=search_dict)
    
    if bids_name_dict:
        bids_name_dict: Dict = deepcopy(bids_name_dict)
    else:
        bids_name_dict: Dict = deepcopy(BIDS_PARAM)
    
    mod_found: bool = False
    
    for i in search_arr:
        if mod_found:
            break
        for k,v in i.items():
            if depth(i) == 3:
                for k2,v2 in v.items():
                    modality_type: str = k
                    modality_label: str = k2
                    task: str = ""
                    mod_search: List[str] = v2
                    if list_in_substr(in_list=mod_search,in_str=s):
                        bids_name_dict: Dict = search_bids(s=s,
                                                           bids_search=bids_search,
                                                           bids_map=bids_map,
                                                           modality_type=modality_type,
                                                           modality_label=modality_label,
                                                           bids_name_dict=bids_name_dict)
            elif depth(i) == 4:
                for k2,v2 in v.items():
                    for k3,v3 in v2.items():
                        modality_type: str = k
                        modality_label: str = k2
                        task: str = k3
                        mod_search: List[str] = v3
                        if list_in_substr(in_list=mod_search,in_str=s):
                            bids_name_dict: Dict = search_bids(s=s,
                                                               bids_search=bids_search,
                                                               bids_map=bids_map,
                                                               modality_type=modality_type,
                                                               modality_label=modality_label,
                                                               task=task,
                                                               bids_name_dict=bids_name_dict)
    return (bids_name_dict,
            modality_type,
            modality_label,
            task)

In [26]:
def batch_proc(subs_data: List[SubDataInfo],
               search_dict: Dict = {},
               meta_dict: Optional[Dict] = {},
               bids_search: Optional[Dict] = {},
               bids_map: Optional[Dict] = {}
              ):
    '''working doc-string'''
    subs_bids: List = []
        
    for sub_data in subs_data:
        data: str = sub_data.data
        bids_name_dict: Dict = deepcopy(BIDS_PARAM)
        bids_name_dict['info']['sub'] = sub_data.sub
        if sub_data.ses:
            bids_name_dict['info']['ses'] = sub_data.ses
        [bids_name_dict, modality_type, modality_label, task] = bids_id(s=data,
                                                                        search_dict=search_dict,
                                                                        bids_search=bids_search,
                                                                        bids_map=bids_map,
                                                                        bids_name_dict=bids_name_dict)
        [meta_com_dict, meta_scan_dict] = get_metadata(dictionary=meta_dict,
                                                       modality_type=modality_type,
                                                       task=task)
        # convert data here

In [27]:
BIDS_PARAM

{'info': {'sub': '', 'ses': ''},
 'anat': {'acq': '', 'ce': '', 'rec': '', 'run': '', 'modality_label': ''},
 'func': {'task': '',
  'acq': '',
  'ce': '',
  'dir': '',
  'rec': '',
  'run': '',
  'echo': '',
  'modality_label': ''},
 'dwi': {'acq': '', 'dir': '', 'run': '', 'modality_label': ''},
 'fmap': {'acq': '',
  'run': '',
  'case1': {'phasediff': '', 'magnitude1': '', 'magnitude2': ''},
  'case2': {'phase1': '', 'phase2': '', 'magnitude1': '', 'magnitude2': ''},
  'case3': {'magnitude': '', 'fieldmap': ''},
  'case4': {'ce': '', 'dir': '', 'modality_label': 'epi'}}}

In [31]:
# Pass the large number of args as a dict
# 
# Needs to run within for-loop
# 
# Unable to test without actual data
# 
# img_data = convert_image_data(file=subs_data[0].data,
#                                 basename="random_str",
#                                 out_dir="out_dir")

In [25]:
def data_to_bids(sub_data: SubDataInfo,
                 bids_name_dict: Dict,
                 out_dir: str,
                 modality_type: Optional[str] = "",
                 modality_label: Optional[str] = "",
                 task: Optional[str] = "",
                 meta_dict: Optional[Dict] = {},
                 mod_dict: Optional[Dict] = {},
                 log: Optional[LogFile] = None,
                 gzip: bool = True,
                 env: Optional[Dict] = {},
                 dryrun: bool = False
                 ):
    '''
    TODO: 
        * Construct dcm2niix's args as a dict, and pass that as args to this function.
    '''
    sub: Union[int,str] = sub_data.sub
    ses: Union[int,str] = sub_data.ses
    data: str = sub_data.data
    
    sub_dir: str = os.path.join(out_dir,"sub-" + sub)
    sub_tmp: str = sub_dir
    
    if ses:
        sub_dir: str = os.path.join(sub_dir,"ses-" + ses)
    
    if sub_dir:
        sub_dir: str = os.path.abspath(sub_dir)
    else:
        os.makedirs(sub_dir)
        sub_dir: str = os.path.abspath(sub_dir)
    
    out_data_dir: str = os.path.join(sub_dir, modality_type)
    
    # construct bids file name
    # Write function to construct BIDS name args,
    #     this function needs to be able handle
    #     python dict KeyErrors
    bids_name_dict = construct_bids_name(sub_data=sub_data,
                                         modality_type=modality_type,
                                         modality_label=modality_label,
                                         acq=bids_name_dict[modality_type][modality_label])
    
    if '.par' in data or '.dcm' in data:
        with TmpDir(tmp_dir=sub_tmp,use_cwd=False) as tmp:
            with TmpDir.TmpFile(tmp_dir=tmp.tmp_dir) as f:
                tmp.mk_tmp_dir()
                [_path, basename, _ext] = f.file_parts()
                try:
                    img_data = convert_image_data(file=data,
                                                  basename=basename,
                                                  out_dir=tmp.tmp_dir,
                                                  log=log,
                                                  env=env,
                                                  dryrun=dryrun,
                                                  return_obj=True)

                    # Update JSON files
                    for i in range(0,len(img_data.imgs)):
                        if img_data.jsons[i]:
                            json_dict: Dict = read_json(json_file=img_data.jsons[i])

                            metadata: Dict = dict_multi_update(**meta_dict,
                                                               **mod_dict)

                            bids_dict: Dict = construct_bids_dict(meta_dict=metadata,
                                                                  json_dict=json_dict)

                            img_data.jsons[i]: str = write_josn(json_file=img_data.jsons[i],
                                                                dictionary=bids_dict)
                    # Create output directory
                    out_data_dir: str = os.path.join(sub_dir, modality_type)
                except ConversionError:
                    pass
                
                # Put modality specific conversion functions here
    elif '.nii' in sub_data.data:
        # Handle NIFTI cases here
        pass
    else:
        return None

In [79]:
# def bids_id(s:str,
#             search_dict: Dict,
#             bids_search: Optional[Dict] = None,
#             bids_map: Optional[Dict] = None,
#             bids_name_dict: Optional[Dict] = None
#            ) -> Union[Dict[str,str],None]:
#     '''
#     TODO:
#         * split the input string so that only the relevant parts of the filename are searched.
#         * contingency function to search image file headers
        
#     Performs identification of descriptive BIDS information relevant for file naming, provided
#     a BIDS search dictionary and a BIDS map dictionary. The resulting information is then placed
#     in nested dictionary of BIDS related descriptive terms.
    
#     Usage example:
#         >>> bids_example_dict = bids_id("ImageFile00001.dcm")
        
#     Arguments:
#         s: Input str/file to be searched.
#         search_dict: Dictionary of modality specific search terms.
#         bids_search: Dictionary of BIDS specific search terms.
#         bids_map: Dictionary of BIDS related terms to be mapped to BIDS search terms.
#         bids_name_dict: Existing BIDS name dictionary.
        
#     Returns:
#         Nested dictionary of BIDS descriptive naming related terms.
#     '''
#     search_arr: List[str] = list_dict(d=search_dict)
    
#     if bids_name_dict:
#         bids_name_dict: Dict = deepcopy(bids_name_dict)
#     else:
#         bids_name_dict: Dict = None
    
#     mod_found: bool = False
    
#     for i in search_arr:
#         if mod_found:
#             break
#         for k,v in i.items():
#             if depth(i) == 3:
#                 for k2,v2 in v.items():
#                     modality_type = k
#                     modality_label = k2
#                     mod_search = v2
#                     if list_in_substr(in_list=mod_search,in_str=s):
#                         bids_name_dict: Dict = search_bids(s=s,
#                                                            bids_search=bids_search,
#                                                            bids_map=bids_map,
#                                                            modality_type=modality_type,
#                                                            modality_label=modality_label,
#                                                            bids_name_dict=bids_name_dict)
#             elif depth(i) == 4:
#                 for k2,v2 in v.items():
#                     for k3,v3 in v2.items():
#                         modality_type = k
#                         modality_label = k2
#                         task = k3
#                         mod_search = v3
#                         if list_in_substr(in_list=mod_search,in_str=s):
#                             bids_name_dict: Dict = search_bids(s=s,
#                                                                bids_search=bids_search,
#                                                                bids_map=bids_map,
#                                                                modality_type=modality_type,
#                                                                modality_label=modality_label,
#                                                                task=task,
#                                                                bids_name_dict=bids_name_dict)
#     return bids_name_dict

In [29]:
subs_data[65].data

'/Users/adebayobraimah/Desktop/projects/convert_source/work_dir/test.img/002-001/PAR REC/FLAIR.PAR'

In [30]:
subs_data[61].data

'/Users/adebayobraimah/Desktop/projects/convert_source/work_dir/test.img/002-001/PAR REC/task-2-func.PAR'

In [31]:
subs_data[66].data

'/Users/adebayobraimah/Desktop/projects/convert_source/work_dir/test.img/002-001/PAR REC/T1.PAR'

In [32]:
subs_data[64].data

'/Users/adebayobraimah/Desktop/projects/convert_source/work_dir/test.img/002-001/PAR REC/DWI_sbref.PAR'

In [70]:
dd = deepcopy(BIDS_PARAM)

In [71]:
dd['info']['sub'] = subs_data[61].sub

In [72]:
dd['info']['ses'] = subs_data[61].ses

In [73]:
dd

{'info': {'sub': '002', 'ses': '001'},
 'anat': {'acq': '', 'ce': '', 'rec': '', 'run': '', 'modality_label': ''},
 'func': {'task': '',
  'acq': '',
  'ce': '',
  'dir': '',
  'rec': '',
  'run': '',
  'echo': '',
  'modality_label': ''},
 'dwi': {'acq': '', 'dir': '', 'run': '', 'modality_label': ''},
 'fmap': {'acq': '',
  'run': '',
  'case1': {'phasediff': '', 'magnitude1': '', 'magnitude2': ''},
  'case2': {'phase1': '', 'phase2': '', 'magnitude1': '', 'magnitude2': ''},
  'case3': {'magnitude': '', 'fieldmap': ''},
  'case4': {'ce': '', 'dir': '', 'modality_label': 'epi'}}}

In [77]:
bids_id(subs_data[66].data,search_dict,bids_search,bids_map,bids_name_dict=dd)

{'info': {'sub': '002', 'ses': '001'},
 'anat': {'acq': '', 'ce': '', 'rec': '', 'run': '', 'modality_label': 'T1w'},
 'func': {'task': '',
  'acq': '',
  'ce': '',
  'dir': '',
  'rec': '',
  'run': '',
  'echo': '',
  'modality_label': ''},
 'dwi': {'acq': '', 'dir': '', 'run': '', 'modality_label': ''},
 'fmap': {'acq': '',
  'run': '',
  'case1': {'phasediff': '', 'magnitude1': '', 'magnitude2': ''},
  'case2': {'phase1': '', 'phase2': '', 'magnitude1': '', 'magnitude2': ''},
  'case3': {'magnitude': '', 'fieldmap': ''},
  'case4': {'ce': '', 'dir': '', 'modality_label': 'epi'}}}

In [42]:
BIDS_PARAM

{'info': {'sub': '', 'ses': ''},
 'anat': {'acq': '', 'ce': '', 'rec': '', 'run': '', 'modality_label': ''},
 'func': {'task': '',
  'acq': '',
  'ce': '',
  'dir': '',
  'rec': '',
  'run': '',
  'echo': '',
  'modality_label': ''},
 'dwi': {'acq': '', 'dir': '', 'run': '', 'modality_label': ''},
 'fmap': {'acq': '',
  'run': '',
  'case1': {'phasediff': '', 'magnitude1': '', 'magnitude2': ''},
  'case2': {'phase1': '', 'phase2': '', 'magnitude1': '', 'magnitude2': ''},
  'case3': {'magnitude': '', 'fieldmap': ''},
  'case4': {'ce': '', 'dir': '', 'modality_label': 'epi'}}}

In [33]:
list_in_substr(in_list=['rsfMR', 'rest', 'FFE', 'FEEPI','rs-func'],
               in_str=subs_data[61].data)

True

In [112]:
list_in_substr(in_list=['T1', 'TFE'], in_str=subs_data[66].data)

True

In [52]:
ss: List[str] = list_dict(d=search_dict)

In [60]:
depth(ss[1])

4

In [61]:
depth(ss[0])

3

In [73]:
ss[0]

{'anat': {'T1w': ['T1', 'T1w', 'TFE'], 'T2w': ['T2', 'T2w', 'TSE']}}